Author:

    Thomas Ulrich
    
partly based on Vavrycuk (2014), DOI: 10.1093/gji/ggu224  
  
In this notebook, we present the most commonly used method, from Michael(1984), to invert stress from focal mechanisms.  
This method relies on the following assumptions:  
  
(1) tectonic stress is uniform  
(2) earthquakes occur on pre-existing faults with varying orientations   
(3) the slip vector points in the direction of shear stress on the fault (Wallace–Bott hypothesis; see Wallace (1951) and Bott(1959)).  
  
4 parameters of the stress tensor $\tau$ can be inverted if the 3 above assumptions are satisfied: 3 angles defining the orientation of the principal stress vectors, and the stress shape ratio, which balance the relative amplitude of the principal stresses, $R= (\sigma_1-\sigma_2)/(\sigma_1-\sigma_3)$ where $\sigma_i$ are the principal stresses. $\sigma_1$ is the maximum principal stress, and because compression is negative, $\sigma_1<\sigma_2<\sigma_3$.  
  
Because the method is unable to recover 2 parameters, 2 further assumptions are made:  
  
(1) the trace $Tr(\tau) = \sigma_1 + \sigma_2 + \sigma_3$ is assumed to be zero  
(2) the stress tensor is normalized.

Let's call **T** the fault traction vector, **N** the unit vector pointing in the direction of the shear traction, **s** the slip vector, the direction of shear motion along the fault, and **n** the fault normal vector. 

![FaultTractionNormal](FaultTractionNormal.png)

The Wallace–Bott hypothesis, which relates **N** with **s**, allow relating $\tau$ with **s**, and therefore with observed focal mechanisms.  
  
In following derivation, we use the Einstein summation convention (an index variable appearing twice implies summation of that term over all the values of the index) and use the kronecker delta, defined by:

$\delta_{mp}=1$ if m=p, 0 if m!=p

We first express **N** in function of $\sigma$ and **n**.
The normal traction (scalar) is:

$\sigma_n = T_i n_i = \sigma_{ij}n_j n_i$

Therefore the shear traction vector $\boldsymbol{\tau}$, (colinear with **N**) is obtained by:

$\boldsymbol{\tau}_i = T_i - \sigma_n n_i = \sigma_{ij} n_j - \sigma_{kj}n_j n_k n_i$

Using, the Kronecker delta definition, we get:

$\sigma_{ij} n_j = \sigma_{kj} n_j \delta_{ik}$

and then:

$\boldsymbol{\tau}_i = \sigma_{kj} n_j (\delta_{ik} - n_i n_k)$

We then apply the Wallace–Bott hypothesis, and get the matrix form:

$A \boldsymbol{t} = \boldsymbol{s}$

where the unknown **t** is the stress tensor written as:
$$t = \begin{bmatrix} 
\sigma_{11} & \sigma_{12} & \sigma_{13} & \sigma_{22} & \sigma_{23} 
\end{bmatrix}^T$$

And **s** is the normalized slip vector.

$$A = \begin{bmatrix} 
n_1(n_2^2+2n_3^2) & n_2(1−2n_1^2) & n_3(1−2n_1^2) & n_1(-n_2^2+n_3^2) & -2n_1 n_2 n_3\\
n_2  (-n_1^2 + n_3^2) & n_1  (1 - 2  n_2^2) & -2  n_1  n_2  n_3 & n_2  (n_1^2 + 2  n_3^2) & n_3  (1 - 2  n_2^2) \\
n_3  (-2  n_1^2 - n_2^2) & -2  n_1  n_2  n_3 & n_1  (1 - 2  n_3^2) & n_3  (-n_1^2 - 2  n_2^2) & n_2  (1 - 2 n_3^2)
\end{bmatrix}$$

If we use the focal mechanisms of K earthquakes, we get a system of $3K$ equations for 5 unknowns.



In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline

In [ ]:
#mydata = ''
mydata = 'NZ'
#mydata = 'Japan2012-2020'
#mydata = 'Japan2000-2010'


if mydata == 'NZ':
    #moment tensor solutions from Geonet are available for download here
    #https://github.com/GeoNet/data/blob/master/moment-tensor/GeoNet_CMT_solutions.csv
    #we download the raw data using:
    #wget https://raw.githubusercontent.com/GeoNet/data/master/moment-tensor/GeoNet_CMT_solutions.csv
    #HEADER
    #       0     1    2       3        4       5   6       7      8    9   10 11 12  13
    #PublicID,Date,Latitude,Longitude,strike1,dip1,rake1,strike2,dip2,rake2,ML,Mw,Mo,CD,NS,DC,Mxx,Mxy,Mxz,Myy,Myz,Mzz,VR,Tva,Tpl,Taz,Nva,Npl,Naz,Pva,Ppl,Paz

    data = np.genfromtxt('GeoNet_CMT_solutions.csv', delimiter=',', skip_header=1)
    lat = data[:,2]
    lon = data[:,3]
    strike = data[:,4]
    dip = data[:,5]
    rake = data[:,6]
    depth = data[:,13]

    #vizualize e.g. using https://www.openstreetmap.org. export->Manually select a different area

    # region around the Kaikoura earthquake
    ids = np.where((lon>172.7)&(lon<174.9)&(lat>-42.7)&(lat<-40.5)&(depth<20))[0]

    #region around the Alpine Fault
    #ids = np.where((lon>166.6)&(lon<170.6)&(lat>-44.6)&(lat<-43.3)&(depth<20))[0]

    #North of Northern island
    #ids = np.where((lon>176.1)&(lon<178.9)&(lat>-40.3)&(lat<-36.8)&(depth>15))[0]
    
    #South of Southern island
    #ids = np.where((lon>163.1)&(lon<169.5)&(lat>-49.1)&(lat<-44.9)&(depth>15))[0]
    
    print(len(ids))
    strike = data[ids,4]
    dip = data[ids,5]
    rake = data[ids,6]
    
elif mydata == 'Japan2000-2010':
    #Downloaded from http://www.isc.ac.uk/iscbulletin/search/fmechanisms/#reviewed
    data = np.genfromtxt('Japan2000_2010.csv', delimiter=',', skip_header=24)
    lat = data[:,4]
    lon = data[:,5]
    depth = data[:,6]
    ids = np.where((lat<38.2)&(lon>141)&(depth>20))
    #ids = np.where(lon>140.5)
    data = data[ids]
    strike = data[:,19]
    dip = data[:,20]
    rake = data[:,21]

elif mydata == 'Japan2012-2020':
    #Downloaded from http://www.isc.ac.uk/iscbulletin/search/fmechanisms/#reviewed
    data = np.genfromtxt('Japan2012_2020.csv', delimiter=',', skip_header=26)
    lat = data[:,4]
    lon = data[:,5]
    depth = data[:,6]
    ids = np.where((lat<38.2)&(lon>141)&(depth>20))
    data = data[ids]
    strike = data[:,19]
    dip = data[:,20]
    rake = data[:,21]
    
else:
    #default file from Stressinverse_1.1.2
    print('using default dataset')
    strike, dip, rake = np.loadtxt("West_Bohemia_mechanisms.dat", skiprows=2, unpack=True)

#Remove nan if any
strike = strike[~np.isnan(strike)]
dip = dip[~np.isnan(dip)]
rake = rake[~np.isnan(rake)]

print(len(strike))
print(strike, dip, rake)

strike0 = np.radians(strike)
dip0 = np.radians(dip)
rake0 = np.radians(rake)

In [ ]:
def computeA(n1, n2, n3):
    # equation 8 in Vavrycuk et al. , GJI. 2014
    A = np.zeros((3, 5))
    n12 = n1 ** 2
    n22 = n2 ** 2
    n32 = n3 ** 2
    A[0, 0] = n1 * (n22 + 2 * n32)
    A[0, 1] = n2 * (1 - 2 * n12)
    A[0, 2] = n3 * (1 - 2 * n12)
    A[0, 3] = n1 * (-n22 + n32)
    A[0, 4] = -2 * n1 * n2 * n3

    A[1, 0] = n2 * (-n12 + n32)
    A[1, 1] = n1 * (1 - 2 * n22)
    A[1, 2] = -2 * n1 * n2 * n3
    A[1, 3] = n2 * (n12 + 2 * n32)
    A[1, 4] = n3 * (1 - 2 * n22)

    A[2, 0] = n3 * (-2 * n12 - n22)
    A[2, 1] = -2 * n1 * n2 * n3
    A[2, 2] = n1 * (1 - 2 * n32)
    A[2, 3] = n3 * (-n12 - 2 * n22)
    A[2, 4] = n2 * (1 - 2 * n32)
    return A

In [ ]:
#Please ignore this function, used to plot the principal stress tensor

#*************************************************************************#
#                                                                         #
#  function PLOT_STRESS_AXES                                              #
#                                                                         #
#  plot of the principal stress axes into the focal sphere                #
#                                                                         #
#  input: principal stress directions                                     #
#         name of figure                                                  #
#  author: Vavrycuk
#                                                                         #
#*************************************************************************#
def plot_stress_axes(sigma_vector_1_,sigma_vector_2_,sigma_vector_3_,plot_file):
    
    import matplotlib.pyplot as plt
    import numpy as np
    import matplotlib

    N = np.size(sigma_vector_1_[1,:])
    
    #--------------------------------------------------------------------------
    # loop over individual solutions
    #--------------------------------------------------------------------------
    x_sigma_1 = np.zeros(N); x_sigma_2 = np.zeros(N); x_sigma_3 = np.zeros(N)
    y_sigma_1 = np.zeros(N); y_sigma_2 = np.zeros(N); y_sigma_3 = np.zeros(N)
    
    for i in range (N):
        
        sigma_vector_1 = sigma_vector_1_[:,i]
        sigma_vector_2 = sigma_vector_2_[:,i]
        sigma_vector_3 = sigma_vector_3_[:,i]
        
        if (sigma_vector_1[2]<0): sigma_vector_1 = -sigma_vector_1; 
        if (sigma_vector_2[2]<0): sigma_vector_2 = -sigma_vector_2; 
        if (sigma_vector_3[2]<0): sigma_vector_3 = -sigma_vector_3; 
        
        #----------------------------------------------------------------------
        # 1. stress axis
        azimuth_sigma_1 =  np.arctan2(sigma_vector_1[1],sigma_vector_1[0])*180/np.pi
        theta_sigma_1 = np.arccos(np.abs(sigma_vector_1[2]))*180/np.pi
        
        #----------------------------------------------------------------------
        # 2. stress axis
        azimuth_sigma_2 =  np.arctan2(sigma_vector_2[1],sigma_vector_2[0])*180/np.pi      
        theta_sigma_2 = np.arccos(np.abs(sigma_vector_2[2]))*180/np.pi
        
        #----------------------------------------------------------------------
        # 3. stress axis
        azimuth_sigma_3 =  np.arctan2(sigma_vector_3[1],sigma_vector_3[0])*180/np.pi         
        theta_sigma_3 = np.arccos(np.abs(sigma_vector_3[2]))*180/np.pi
        print(azimuth_sigma_1,azimuth_sigma_2,azimuth_sigma_3)
        #----------------------------------------------------------------------
        # projection into the lower hemisphere
        projection_1 = 1; projection_2 = 1; projection_3 = 1;
            
        #----------------------------------------------------------------------
        #  zenithal equal-area projection
        
        radius_sigma_1 = projection_1*np.sin(theta_sigma_1*np.pi/360.)
        radius_sigma_2 = projection_2*np.sin(theta_sigma_2*np.pi/360.)
        radius_sigma_3 = projection_3*np.sin(theta_sigma_3*np.pi/360.)
        
        x_sigma_1[i] = np.sqrt(2.)*radius_sigma_1*np.cos(azimuth_sigma_1*np.pi/180.)
        y_sigma_1[i] = np.sqrt(2.)*radius_sigma_1*np.sin(azimuth_sigma_1*np.pi/180.)
        
        x_sigma_2[i] = np.sqrt(2.)*radius_sigma_2*np.cos(azimuth_sigma_2*np.pi/180.)
        y_sigma_2[i] = np.sqrt(2.)*radius_sigma_2*np.sin(azimuth_sigma_2*np.pi/180.)
        
        x_sigma_3[i] = np.sqrt(2.)*radius_sigma_3*np.cos(azimuth_sigma_3*np.pi/180.)
        y_sigma_3[i] = np.sqrt(2.)*radius_sigma_3*np.sin(azimuth_sigma_3*np.pi/180.)
    
    #--------------------------------------------------------------------------
    # plotting the stress directions in the focal sphere
    #--------------------------------------------------------------------------
    plotSA, axSA = plt.subplots()
    
    plt.title('principal stress axes into the focal sphere',fontsize = 16);
    axSA.axis ('equal')
    axSA.axis([-1.05, 1.70, -1.05, 1.05])
    axSA.axis('off')
    
    
    #--------------------------------------------------------------------------
    # plotting the stress directions
    sig1, = plt.plot(y_sigma_1,x_sigma_1,'r.', markersize = 20);	# sigma_1
    sig2, = plt.plot(y_sigma_2,x_sigma_2,'g.', markersize = 20);	# sigma_2
    sig3, = plt.plot(y_sigma_3,x_sigma_3,'b.', markersize = 20);	# sigma_3
       
    #--------------------------------------------------------------------------
    # boundary circle and the centre of the circle
    fi = np.arange(0, 360, 0.1)		
    plt.plot(np.cos(fi*np.pi/180.),np.sin(fi*np.pi/180.),'k-', linewidth = 2.0)
    plt.plot(0,0,'k+', markersize = 10);		
    
    #--------------------------------------------------------------------------
    # grid lines - constant theta
    theta_grid_i = np.arange(0, 90, 15)
    for theta_grid in (theta_grid_i) :
        radius_grid = projection_1*np.sin(theta_grid*np.pi/360.)
        
        x_grid = np.sqrt(2.)*radius_grid*np.cos(fi*np.pi/180.)
        y_grid = np.sqrt(2.)*radius_grid*np.sin(fi*np.pi/180.)
    
        plt.plot(y_grid,x_grid,'k:', linewidth = 0.5)
    
    #--------------------------------------------------------------------------
    # grid lines - constant fi
    theta_grid = np.arange(0, 105, 15)
    
    fi_grid_i = np.arange(0,360,15)
    for fi_grid in (fi_grid_i) :
        radius_grid = projection_1*np.sin(theta_grid*np.pi/360.)
        
        x_grid = np.sqrt(2.)*radius_grid*np.cos(fi_grid*np.pi/180.)
        y_grid = np.sqrt(2.)*radius_grid*np.sin(fi_grid*np.pi/180.)
    
        plt.plot(y_grid,x_grid,'k:', linewidth = 0.5);
    
    #--------------------------------------------------------------------------
    # legend
    axSA.legend((sig1,sig2,sig3), ('sigma 1','sigma 2','sigma 3'), loc = 'lower right', fontsize = 14, numpoints=1)
    if plot_file=='':
       plt.show()
    else:
       plt.savefig(plot_file + '.png')
       plt.close()

In [ ]:
def ComputeSlipVector(strike, dip, rake):
    # Compute slip vector from focal mechanisms
    nobservations = strike.shape[0]
    SlipVector = np.zeros((nobservations, 3))

    # e.g. Stein and Wyssession (p218)
    SlipVector[:, 0] = np.cos(rake[:]) * np.cos(strike[:]) + np.sin(rake[:]) * np.cos(dip[:]) * np.sin(strike[:])
    SlipVector[:, 1] = -np.cos(rake[:]) * np.sin(strike[:]) + np.sin(rake[:]) * np.cos(dip[:]) * np.cos(strike[:])
    SlipVector[:, 2] = np.sin(rake[:]) * np.sin(dip[:])

    return SlipVector.flatten()

def ComputeNormalVector(strike, dip, rake):
    # Compute fault normal from focal mechanisms
    nobservations = strike.shape[0]
    FaultNormal = np.zeros((nobservations, 3))
    # e.g. Stein and Wyssession (p218)
    FaultNormal[:, 0] = -np.sin(dip[:]) * np.sin(strike[:])
    FaultNormal[:, 1] = -np.sin(dip[:]) * np.cos(strike[:])
    FaultNormal[:, 2] = np.cos(dip[:])

    return FaultNormal

def AssembleAmatrix(FaultNormal):
    nobservations = FaultNormal.shape[0]
    # Assemble A matrix
    A = np.zeros((3 * nobservations, 5))
    for i in range(nobservations):
        A[3 * i : 3* i + 3, :] = computeA(*FaultNormal[i, :])
    return A

def computeStressTensor(A, S):
    # Solve the Problem At=s using the generalized inverse of A
    # A^-g = (A^t A)^-1 A^t (see e.g. Stein and Wyssession (p418, eq 17)
    # this the best solution in the least square sense
    Ag = A.T @ A
    Ag_inv = np.linalg.inv(Ag)
    t = Ag_inv @ A.T @ S

    ## t indices are 11, 12, 13, 22, 23
    ## stress tensor in ENU
    #Tij = np.array([[t[0], t[1], t[2]], [t[1], t[3], t[4]], [t[2], t[4], -t[0]-t[3]]])
    ## stress tensor in ESD
    Tij = np.array([[t[0], -t[1], -t[2]], [-t[1], t[3], t[4]], [-t[2], t[4], -t[0]-t[3]]])
    return Tij

def ComputeSortedEigenValuesAndVectors(Tij):
    #Compute eigen Values and eigen Vectors, to compute R ratio and SH_max

    #each column of eigenVectors is an eigenvector
    eigenValues, eigenVectors = np.linalg.eig(Tij)
    #important notes: 
    # the largest principal stress corresponds with the smallest seigeValue!
    # the eigenValues should not be sorted in absolute value!
    idx = eigenValues.argsort()
    eigenValues = eigenValues[idx]
    eigenVectors = eigenVectors[:, idx]
    return eigenValues, eigenVectors


In [ ]:
nrealization = 30

eigenValues = np.zeros((nrealization, 3))
eigenVectors = np.zeros((nrealization, 3,3))

for irealization in range(nrealization):

    if nrealization>1:
        nobservations = strike0.shape[0]
        listn = np.random.choice(range(nobservations), size=int(0.8*nobservations), replace=False)
        strike = strike0[listn]
        dip = dip0[listn]
        rake = rake0[listn]
    else:
        strike=strike0
        dip = dip0
        rake = rake0
    
    FaultNormal = ComputeNormalVector(strike, dip, rake)
    S = ComputeSlipVector(strike, dip, rake)
    A = AssembleAmatrix(FaultNormal)
    Tij = computeStressTensor(A, S)
    eigenValues[irealization, :], eigenVectors[irealization, :, :] = ComputeSortedEigenValuesAndVectors(Tij)

#print('stress tensor in ESD\n', Tij)
#print("sorted eigen values", eigenValues)
#print("sorted eigen vectors\n", eigenVectors)

#plot the eigen vector in the focal sphere
v1 = (eigenVectors[:,:,0]).T
v2 = (eigenVectors[:,:,1]).T
v3 = (eigenVectors[:,:,2]).T
plot_stress_axes(v1, v2, v3, '')

In [ ]:
def ComputeSHmax(eigenVectors):
    # largest principal stress
    uz = np.array([0.0, 0.0, 1.0])
    idv = int(np.argmax(np.absolute(np.dot(eigenVectors.T, uz))))
    #print("most vertical vector", idv + 1)

    # compute largest eigen value not the most vertical eigen vector
    if idv == 0:
        ih = 1
    else:
        ih = 0
        
    vh = eigenVectors[:, ih]
    # SHmax: azimuth of the maximum horizontal compressive stress
    SHmax = np.degrees(np.arctan2(vh[1], vh[0]))
    return SHmax 


    
def plotSHmaxR(eigenVectors, eigenValues):
    #Compute SH_max and R (s2_ratio)
    R = (eigenValues[:,0] - eigenValues[:,1]) / (eigenValues[:,0] - eigenValues[:,2])
    plt.title('R distribution',fontsize = 16);
    plt.hist(R)
    plt.show()
    nobservations = eigenValues.shape[0]
    SHmax = np.zeros(nobservations)
    for i in range(nobservations):
        SHmax[i] = ComputeSHmax((eigenVectors[i,:,:]).T)
    plt.title('SHmax distribution',fontsize = 16);
    plt.hist(SHmax)
    plt.show()    

In [ ]:
plotSHmaxR(eigenVectors, eigenValues)